# Subjective Quiz | Big Data on Spark

Dataset: https://archive.ics.uci.edu/ml/datasets/Wholesale+customers#

The dataset refers to clients of a wholesale distributor. It includes the annual spending in monetary units (m.u.) on diverse product categories

Variables:

1.  FRESH: annual spending (m.u.) on fresh products (Continuous);
2.  MILK: annual spending (m.u.) on milk products (Continuous);
3.  GROCERY: annual spending (m.u.)on grocery products (Continuous);
4.  FROZEN: annual spending (m.u.)on frozen products (Continuous)
5.  DETERGENTS_PAPER: annual spending (m.u.) on detergents and paper products (Continuous)
6.  DELICATESSEN: annual spending (m.u.)on and delicatessen products (Continuous);
7.  CHANNEL: customers Channel - Horeca (Hotel/Restaurant/Cafe) or Retail channel (Nominal)
8.  REGION: customers Region Lisbon, Oporto or Other (Nominal)

Goal:
Learn DataFrame Operations in Spark

## Load Libraries

In [20]:
from pyspark import SparkContext,SQLContext,SparkConf,StorageLevel
import numpy as np
from pyspark.sql.functions import *

## SparkContext and SparkConfiguration

In [21]:
# setMaster("local[3]") says it's local driver with 3 
sparkconf= SparkConf().setAppName("SubQuizMP").setMaster("local[3]")
sc= SparkContext(conf=sparkconf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=SubQuizMP, master=local[3]) created by __init__ at <ipython-input-4-06bfe0a9bdbb>:3 

In [7]:
sqlContext = SQLContext(sc)

## 1. Read the csv file as a dataframe. And, not as RDD. See the schema of the DF.

In [9]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('Wholesalecustomersdata.csv')

print("Schema of the df:")
print(df.dtypes)

Schema of the df:
[('Channel', 'int'), ('Region', 'int'), ('Fresh', 'int'), ('Milk', 'int'), ('Grocery', 'int'), ('Frozen', 'int'), ('Detergents_Paper', 'int'), ('Delicassen', 'int')]


## 2. Use select to view a single column or a set of chosen columns.

In [11]:
df.select(df['Channel']).show()

+-------+
|Channel|
+-------+
|      2|
|      2|
|      2|
|      1|
|      2|
|      2|
|      2|
|      2|
|      1|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      1|
|      2|
|      1|
|      2|
|      1|
+-------+
only showing top 20 rows



In [12]:
df.select(df['Region'],df['Fresh'],df['Milk'],df['Grocery'],df['Frozen']).show()

+------+-----+-----+-------+------+
|Region|Fresh| Milk|Grocery|Frozen|
+------+-----+-----+-------+------+
|     3|12669| 9656|   7561|   214|
|     3| 7057| 9810|   9568|  1762|
|     3| 6353| 8808|   7684|  2405|
|     3|13265| 1196|   4221|  6404|
|     3|22615| 5410|   7198|  3915|
|     3| 9413| 8259|   5126|   666|
|     3|12126| 3199|   6975|   480|
|     3| 7579| 4956|   9426|  1669|
|     3| 5963| 3648|   6192|   425|
|     3| 6006|11093|  18881|  1159|
|     3| 3366| 5403|  12974|  4400|
|     3|13146| 1124|   4523|  1420|
|     3|31714|12319|  11757|   287|
|     3|21217| 6208|  14982|  3095|
|     3|24653| 9465|  12091|   294|
|     3|10253| 1114|   3821|   397|
|     3| 1020| 8816|  12121|   134|
|     3| 5876| 6157|   2933|   839|
|     3|18601| 6327|  10099|  2205|
|     3| 7780| 2495|   9464|   669|
+------+-----+-----+-------+------+
only showing top 20 rows



## 3. Use filter to see records with fresh sales more than 50000 only.

In [22]:
df.filter(df.Fresh > 50000).show()

+-------+------+------+-----+-------+------+----------------+----------+
|Channel|Region| Fresh| Milk|Grocery|Frozen|Detergents_Paper|Delicassen|
+-------+------+------+-----+-------+------+----------------+----------+
|      1|     3| 56159|  555|    902| 10002|             212|      2916|
|      1|     3| 56082| 3504|   8906| 18028|            1480|      2498|
|      1|     3| 76237| 3473|   7102| 16538|             778|       918|
|      1|     3|112151|29627|  18148| 16745|            4948|      8550|
|      1|     1| 56083| 4563|   2124|  6422|             730|      3321|
|      1|     1| 53205| 4959|   7336|  3012|             967|       818|
|      1|     3| 68951| 4411|  12609|  8692|             751|      2406|
+-------+------+------+-----+-------+------+----------------+----------+



## 4. Create aggregates on channels and regions variables.

In [24]:
gdf = df.groupBy(df.Channel)
df_Channel = sorted(gdf.agg({"*": "count"}).collect())
df_Channel

[Row(Channel=1, count(1)=298), Row(Channel=2, count(1)=142)]

In [25]:
gdf = df.groupBy(df.Region)
df_Region = sorted(gdf.agg({"*": "count"}).collect())
df_Region

[Row(Region=1, count(1)=77),
 Row(Region=2, count(1)=47),
 Row(Region=3, count(1)=316)]

## 5. Use describe to see summary statistics on dataframe.

In [15]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+
|summary|           Channel|            Region|             Fresh|              Milk|          Grocery|           Frozen|  Detergents_Paper|        Delicassen|
+-------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+
|  count|               440|               440|               440|               440|              440|              440|               440|               440|
|   mean|1.3227272727272728| 2.543181818181818|12000.297727272728| 5796.265909090909|7951.277272727273|3071.931818181818|2881.4931818181817|1524.8704545454545|
| stddev|0.4680515694791137|0.7742724492301002|12647.328865076885|7380.3771745708445|9503.162828994346|4854.673332592367| 4767.854447904201|2820.1059373693965|
|    min|                 1|            

## 6. Change datatype of Channels to Strings.

In [16]:
df1 = df.withColumn('Channel',df.Channel.cast('String'))

In [18]:
df1.dtypes

[('Channel', 'string'),
 ('Region', 'int'),
 ('Fresh', 'int'),
 ('Milk', 'int'),
 ('Grocery', 'int'),
 ('Frozen', 'int'),
 ('Detergents_Paper', 'int'),
 ('Delicassen', 'int')]

In [19]:
df1.select(df1['Channel']).show()

+-------+
|Channel|
+-------+
|      2|
|      2|
|      2|
|      1|
|      2|
|      2|
|      2|
|      2|
|      1|
|      2|
|      2|
|      2|
|      2|
|      2|
|      2|
|      1|
|      2|
|      1|
|      2|
|      1|
+-------+
only showing top 20 rows

